In [1]:
#!pip install transformers faiss-cpu datasets torch sentence_transformers
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: C:\Users\igorm\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [3]:
#https://chatgpt.com/c/3d820dd3-8354-4b8b-b86d-1b78caa32d93

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU disponivel')
    print('Número de GPUs disponíveis:', torch.cuda.device_count())
    print('Nome da GPU:', torch.cuda.get_device_name(0))

else:
    device = torch.device('cpu')
    print('GPU nao disponivel, utilizando CPU')

GPU disponivel
Número de GPUs disponíveis: 1
Nome da GPU: NVIDIA GeForce RTX 2070


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.cuda.amp import autocast




tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

c:\Users\igorm\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:42<00:00, 10.63s/it]


In [2]:
import json

data = []
with open('financial_market_qa.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

inputs = [item['input'] for item in data]
outputs = [item['output'] for item in data]

In [3]:
from transformers  import AutoTokenizer, AutoModel
import torch
import numpy as np

model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu()


question_embeddings = np.array([get_embedding(question).numpy().flatten() for question in inputs])

c:\Users\igorm\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
import faiss

d = question_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(question_embeddings)


In [5]:
def retrieve_relevant_answers(query, k=5):
    # k = quantidade de respostas relevantes retornadas
    query_embedding = get_embedding(query).numpy().flatten().reshape(1, -1)
    distances, indices = index.search(query_embedding, k)
    return [outputs[i] for i in indices[0]]

In [6]:
def generate_response(query):
    relevant_answers = retrieve_relevant_answers(query)
    context = " ".join(relevant_answers) + " " + query
    inputs = tokenizer(context, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

: 

In [7]:
response = generate_response("O que é o mercado financeiro?")

print(response)